In [21]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import glob
import numpy as np
import cv2
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from ket_utilityP4 import *
%matplotlib inline


In [22]:
def get_calibration_parameters():
    #get the camera calibration parameters
    #either from the file or from image processing 
    if (os.path.isfile('P4CameraParam.p'))==True:
        # read python dict back from the file
        print('Reading Camera Parameters from file')
        pickle_file = open('P4CameraParam.p', 'rb')
        p4dict = pickle.load(pickle_file)
        ret=p4dict['ret']
        mtx=p4dict['mtx']
        dist=p4dict['dist']
        rvecs=p4dict['rvecs']
        tvecs=p4dict['tvecs']
        nx=p4dict['nx']
        nx=p4dict['ny']
        pickle_file.close()
    else:
        print('Camera Param file not found!!')
        #number of corners in x and y directions
        nx=9
        ny=6
        #read the images
        cal_files='./camera_cal/calibration*.jpg'
        image_files=glob.glob(cal_files)
        #just to get image size
        dummy_img=cv2.imread('./camera_cal/calibration1.jpg')
        img_size=(dummy_img.shape[1], dummy_img.shape[0])
        #get the image and object points using utility function from ket_utilityP4
        imgpoints, objpoints=get_imagepoints_objpoints(image_files, gridsize=(nx, ny), debug_prt=0)
        ret, mtx, dist, rvecs, tvecs=cv2.calibrateCamera(objpoints, imgpoints,img_size, None, None)
        p4dict = {'ret': ret, 'mtx': mtx, 'dist': dist, 'rvecs': rvecs, 'tvecs': tvecs, 'nx': nx, 'ny': ny}
        output = open('P4CameraParam.p', 'wb')

        pickle.dump(p4dict, output)
        output.close()
    
    return mtx, dist

In [23]:
#Slight modifications to provided pipeline.
def pipeline(img):
    # Gaussian Blur
    kernel_size = 7
    img = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    # S channel 
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s = hls[:,:,2]
    # Grayscale image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Sobel kernel size
    ksize = 7
    # Thresholding functions for each case
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(10, 255))
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(60, 255))
    mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(40, 255))
    dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(.65, 1.05))
    # Combine all the thresholded images
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    # Threshold color channel
    s_binary = np.zeros_like(combined)
    s_binary[(s > 160) & (s < 255)] = 1
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors    
    color_binary = np.zeros_like(combined)
    color_binary[(s_binary > 0) | (combined > 0)] = 1
    
    return color_binary
    

In [24]:
#Warped image parameters for the given source corners
#Finding the right corners is important
def get_M_Minv():
    #corner of the source
    left_bottom=[40,680]
    left_top=[490,482]
    right_top=[810,482]
    right_bottom=[1250,680]
    #source
    src=np.float32([left_top, right_top, right_bottom, left_bottom])
    #now destination
    #slight skew in the rectangle
    dst=np.float32([[0,0], [1280,0], [1250, 720],[40,720]])
    #perspective transportmation  
    M=cv2.getPerspectiveTransform(src, dst)
    Minv=cv2.getPerspectiveTransform(dst, src)
    
    return M, Minv 

In [25]:
#output of pipeline is fed to this function
# code is from mostly Module 33
def get_lines (binary_warped):
    #most of this directly from Module 33 
    histogram = np.sum(binary_warped[:,:], axis=0)
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    #margin = 100
    margin=100
    # Set minimum number of pixels found to recenter window
    #minpix=50
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(binary_warped,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(binary_warped,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    '''
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    '''
    
    return left_fitx, right_fitx, ploty


In [26]:
# Create an image to draw the lines on
def unwrap(undistorted_img, left_fitx, right_fitx, ploty, Minv):
    
    im_size=(undistorted_img.shape[1], undistorted_img.shape[0])
    color_warp = np.zeros_like(undistorted_img).astype(np.uint8)
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, im_size)
    
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted_img, 1, newwarp, 0.3, 0)
    
    return result
    

In [27]:
#mainly used Module 35
def get_curvature(leftx, rightx, ploty):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    y_eval = np.max(ploty)

    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Return the max of the two
    return left_curverad, right_curverad

In [28]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [29]:
#sanity check methods
def vehicle_lane_offset(left_fitx, right_fitx):
    #assuming the vehicle is in the middle of the x axis pixel 640
    #compare the location of the vehicle to the middle lane pixels at Y=640.
    left=np.mean(left_fitx[0:40])
    right=np.mean(right_fitx[0:40])
    
    centered=(right+left)/2.0
    xm_per_pix = 3.7/700   
    offset=(640-centered)*xm_per_pix
    
    return offset     

def curvature_check():
    #check if the consecutive curvatures are simular at least with in 25% change.
    return None

    
def if_lines_parallel():
    #check to see if the lines are parallel
    return None
    
def sanity_check_reset():
    #if sanity checks fails, reset for restart
    return None

def smoothing():
    #smooth the lines
    return None

def look_ahead_filter():
    #search from previously detected frames
    return None

In [35]:
#no need to get M and Ninv
#and mtc, dist for each frame.

M, Minv=get_M_Minv()
mtx, dist= get_calibration_parameters()

def frame_process(image):
    #get image size
    img_size=(image.shape[1], image.shape[0])
    #undistort the image
    undistorted_img = cv2.undistort(image, mtx, dist, None, mtx)
    #get warped image
    warped = cv2.warpPerspective(undistorted_img, M, img_size, flags=cv2.INTER_LINEAR)
    #get the lines in binary
    binary_warped=pipeline(warped)
    #get fitxs' and ploty
    left_fitx, right_fitx, ploty=get_lines(binary_warped)
    #check the curvature
    curve_radius=get_curvature(left_fitx, right_fitx, ploty)
    #print('curvature', curve_radius)
    #uwrap and combine everything
    result=unwrap(undistorted_img, left_fitx, right_fitx, ploty, Minv)
    #get offset
    offset=vehicle_lane_offset(left_fitx, right_fitx)
    cv2.putText(result, 'Offset {:.2f}m -left, +right'.format(offset), (100,80), fontFace=16, fontScale=2, color=(255,0,0), thickness=2)
    cv2.putText(result, 'Avg curvature {:.2f}m'.format(np.mean(curve_radius)), (120,80), fontFace=16, fontScale=2, color=(255,0,0), thickness=2)       
    
    return result


Reading Camera Parameters from file


In [36]:
output_video='P4_ket_out.mp4'
clip1=VideoFileClip("project_video.mp4")
white_clip=clip1.fl_image(frame_process)
white_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video P4_ket_out.mp4
[MoviePy] Writing video P4_ket_out.mp4



100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [08:19<00:00,  2.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: P4_ket_out.mp4 



In [ ]:
HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(output_video))

In [ ]:
output_video='P4_ket_out_challenge.mp4'
clip1=VideoFileClip("challenge_video.mp4")
white_clip=clip1.fl_image(frame_process)
white_clip.write_videofile(output_video, audio=False)

In [ ]:
HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(output_video))